In [10]:
import requests
import json

URI = 'https://api.cian.ru/search-offers/v2/search-offers-desktop/'
headers = {
    'Accept': 'application/json',
    'Content-Type': 'application/json',
}
data = {
    "jsonQuery": {
        "region":
        {"type": "terms",
         "value": [4704]
         },
        "_type": "commercialsale",
#         'category': {'type': 'terms', 'value': ['commercialLandSale']},
#         'category': {'type': 'terms', 'value': ['commercialLandSale', 'buildingSale', 'officeSale', 'freeAppointmentObjectSale', 'shoppingAreaSale']},
        "office_type": {
            "type": "terms",
            "value": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
        },
        "engine_version": {
            "type": "term",
            "value": 2
        },
        "page": {
            "type": "term",
            "value": 1
        }
    }
}
response = requests.post(URI, headers=headers, data=json.dumps(data))

In [11]:
from data import data_from_dict
result = data_from_dict(response.json())

In [26]:
pages = result.data.aggregated_count // len(result.data.offers_serialized) + 2
pages

43

In [27]:
import pymongo
from pymongo import MongoClient
client = MongoClient('mongodb://localhost:27017/')
db = client.cian
collection = db.offers
collection.create_index([('id', pymongo.ASCENDING)], unique=True)

'id_1'

In [28]:
for offer in result.data.offers_serialized:
    full_offer = offer.to_dict()
    filtered_offer = {k: v for k, v in full_offer.items() if v is not None}
    collection.find_one_and_replace(
        {"id": offer.id}, filtered_offer, upsert=True)

In [13]:
with open("output.txt", "a+") as text_file:
    print(response.text, file=text_file)

In [ ]:
for x in range (2,pages):
    data['page']['value'] = x
    response = requests.post(
        'https://www.cian.ru/cian-api/mobile-site/v2/search-offers/', headers=headers, data=json.dumps(data))
    with open("output.txt", "a+") as text_file:
        print(response.text, file=text_file)
    result = data_from_dict(response.json())
    print(result.data.offers_serialized)